# Streaming

Streaming is a capability that allows AI providers to deliver responses as tokens are generated in real-time,
rather than waiting for the complete response to be finished.
You can visualize it as filling a bucket with water — streaming lets you watch the water level rising in real-time,
instead of placing the bucket somewhere, walking away, and only returning once it's completely full.

Streaming significantly improves the user experience,
as people can immediately see the response forming rather than staring at a loading indicator.
This is especially valuable since synchronous requests can take quite a long time to generate.

Let's implement streaming responses with Kotlin and Spring AI.
For this, we'll need to add dependencies.
In addition to `spring-ai-openai`, we'll need the Kotlin coroutines library to work with `Flow`.

In [1]:
%useLatestDescriptors
%use coroutines
%use spring-ai-openai
USE { dependencies { implementation("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.10.2") } }

As in the previous notebooks, we'll need an API key.

To use the model, we need to provide an API key.

You can obtain this API key from
[console.anthropic.com](https://console.anthropic.com/settings/keys)
for Anthropic models or from
[platform.openai.com](https://platform.openai.com/api-keys)
for OpenAI models.

Then add the generated API key to your environment variables:

[MacOS/Linux]
```bash
export ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
export OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI

```

[Windows]
```shell
set ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
set OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI
```

Let's retrieve the API key from environment variables:

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

Now let's create a `ChatModel` and pass in the `ChatOptions`:

In [3]:
val openAiApi = OpenAiApi.builder().apiKey(apiKey).build()
val openAiOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O_MINI)
    .temperature(0.7)
    .build()

val chatCompletion = OpenAiChatModel.builder()
    .openAiApi(openAiApi)
    .defaultOptions(openAiOptions)
    .build()

Now let's ask the model to generate a short fairy tale about rabbits:

In [4]:
import kotlinx.coroutines.reactive.asFlow

val response: Flow<String> = chatCompletion.stream("Generate me 10 sentences of a fairy tale about rabbits").asFlow()

runBlocking {
    response.collect { print(it) }
}

Once upon a time in a lush, green forest, there lived a clever rabbit named Benny who dreamed of becoming the fastest runner in all the land. Benny challenged the boastful tortoise, Timmy, to a race, confident that his speed would secure him victory. As the race began, Benny sprinted ahead, leaving Timmy far behind, and decided to take a nap under a shady tree. Meanwhile, Timmy kept plodding along, steady and determined, never once stopping to rest. 

When Benny awoke, he panicked and dashed towards the finish line, but it was too late; Timmy had crossed it triumphantly, proving that perseverance beats arrogance. The animals of the forest celebrated Timmy's victory with a grand feast, inviting Benny to join and learn from his mistake. Realizing the importance of humility, Benny promised to train harder and respect his fellow creatures' strengths. From that day on, he and Timmy became the best of friends, often racing together but always encouraging one another. 

As the seasons changed

As you can see, the response starts printing immediately.
If this weren't a streaming response,
we would be waiting approximately 8 seconds (judging by the cell execution time) with nothing happening.

Now let's do the same thing using the `ChatClient`:

In [5]:
val chatClient = ChatClient.create(chatCompletion)

val response: Flow<String> = chatClient
    .prompt("Generate me 10 sentences of a fairy tale about rabbits")
    .stream()
    .content()
    .asFlow()

runBlocking {
    response.collect { print(it) }
}

Once upon a time in a lush, green meadow, there lived a clever rabbit named Ruby who dreamed of discovering the hidden treasures of the Enchanted Forest. One sunny morning, Ruby decided to gather her friends — Benny the brave bunny, and Lily the wise old tortoise — to embark on an adventure. As they hopped along the winding path, they stumbled upon a sparkling stream filled with shimmering fish that danced like stars in the water. 

Suddenly, a mischievous fox appeared, claiming to know the secret to finding the treasure, but only if Ruby could solve his riddle. With her quick wit, Ruby answered the riddle, impressing the fox, who reluctantly decided to guide them deeper into the forest. The trio encountered magical creatures along the way, including a singing bird with feathers of gold and a gentle deer who offered them guidance. 

After crossing a bridge made of rainbows, they finally reached a hidden glade where an ancient oak tree stood, its trunk wrapped in vines and glowing with 